In [ ]:
from jitcdde import jitcdde, y, t
from parameters import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# energy flow functions
def dQ_bf(W,c,a,b):
    '''
    Energy from bulk flow
    W: mass flow rate (kg/s)
    c: specific heat capacity (J/(kg*K))
    a: Q from node (state variable y(i))
    b: Q to node (state variable y(i))
    '''
    return (a-b)*W*c

def dQ_int(k,P,n):
    '''
    Energy from fission
    k: fractional adjustment
    P: nominal power (MW)
    n: fractional neutron density 
    '''
    return k*P*n

def dQ_cht(a: list, b, hA: list, f: list = [1.0]*len(a)):
    '''
    Energy from convective heat transfer
    a: from node(s) (state variable(s) y(i))
    b: to node (state variable y(i))
    hA: convective heat transfer coefficient(s) * wetted area(s) (MW/C)
    f: fractional adjustments
    '''
    tot = 0.0
    for i in range(len(a)):
        tot += hA[i]*(a[i]-b)
    return tot


# core fuel nodes
Q_c_f1 = y(0)
Q_c_f2 = y(1)

# core fuel tubes
Q_c_t1 = y(2)

# core stationary coolant node
Q_c_cs = y(3)

# core coolant nodes
Q_c_c1 = y(4)
Q_c_c2 = y(5)

# fuel-helium hx nodes
Q_hfh_f2 = y(6)

n = y(25)

# delaye variables
Q_c_f_in = y(6,t-tau_hx_c_f)

In [ ]:
# CORE
# fuel nodes (divide by mcp_f_c for temperature)
dQ_c_f1 = dQ_bulk_flow(W_f,scp_f,Q_c_f_in,Q_c_f1)+ dQ_internal(k_f1,P,n) + dQ_cht([Q_c_t1],Q_c_f1,[hA_ft_c])
dQ_c_f2 = dQ_bulk_flow(W_f,scp_f,Q_c_f1,Q_c_f2)  + dQ_internal(k_f2,P,n) + dQ_cht([Q_c_t1],Q_c_f2,[hA_ft_c])

# fuel tube node (divide by mcp_t_c for temperature)
dQ_c_t1 = dQ_cht([Q_c_f1,Q_c_f2,Q_c_c1,Q_c_c2],Q_c_t1,[hA_ft_c,hA_ft_c,hA_tc_c,hA_t_c])

# coolant nodes CONTINUE HERE
dQ_c_c1 = dQ_bulk_flow(W_c,scp_c,Q_c_f_in,Q_c_f1)+ dQ_cht([Q_c_t1],Q_c_f1,[hA_ft_c])
dQ_c_c2 = dQ_bulk_flow(W_c,scp_c,Q_c_f1,Q_c_f2)  + dQ_cht([Q_c_t1],Q_c_f2,[hA_ft_c])

Define system

In [ ]:
# %%
# CORE
# fuel nodes
T_c_f1 = W_f/(m_f_c)*(y(6,t-tau_hx_c_f)-y(0)) + (k_f1*P*y(25)/mcp_f_c) + (hA_ft_c*k_1*(y(2)-y(0))/mcp_f_c)# T_cf1: y(0)
T_c_f2 = W_f/(m_f_c)*(y(0)-y(1)) + (k_f2*P*y(25)/mcp_f_c) + (hA_ft_c*k_2*(y(2)-y(1))/mcp_f_c)             # T_cf2: y(1)

# tubes
T_c_t1=(hA_ft_c/mcp_t_c)*((y(0)-y(2))+(y(1)-y(2)))+(hA_tc_c/mcp_t_c)*((y(3)-y(2))+(y(4)-y(2)))      # T_c_t1: y(2)

# coolant 
T_c_c1 = W_c/m_c_c*(y(11,t-tau_hx_c_c)-y(3)) + (hA_tc_c*k_1*(y(2)-y(3))/mcp_c_c)                               # T_c_c1: y(3)
T_c_c2 = W_c/m_c_c*(y(3)-y(4)) + (hA_tc_c*k_2*(y(2)-y(4))/mcp_c_c)                                             # T_c_c1: y(4)  

# moderator 
T_c_b = (hA_ft_c/mcp_t_c)*((y(0)-y(2))+(y(1)-y(2)))+(hA_tc_c/mcp_t_c)*((y(3)-y(2))+(y(4)-y(2))) 

# FUEL-HELIUM HX
# fuel nodes
T_hfh_f1 = W_f/m_f_hx*(y(1,t-tau_c_hx_f)-y(1)) + (hA_ft_hx*k_1*(y(7)-y(5))/mcp_f_hx)                          # T_cf1: y(5)
T_hfh_f2 = W_f/m_f_hx*(y(5)-y(6)) + (hA_ft_hx*k_2*(y(7)-y(6))/mcp_f_hx)                                       # T_cf2: y(6)

# tubes
T_hfh_t1=(hA_ft_hx/mcp_t_hx)*((y(5)-y(7))+(y(6)-y(7)))+(hA_th_hx/mcp_t_hx)*((y(8)-y(7))+(y(9)-y(7)))             # T_ht1: y(7)

# helium
T_hfh_h1 = W_hfh_h1/(m_h_hxfh)*(y(16,t-tau_hx_c_c)-y(8)) + (hA_th_hx*k_1*(y(7)-y(8))/mcp_h_hxfh)                # T_cc1: y(8) 
T_hfh_h2 = W_hfh_h2/(m_h_hxfh)*(y(8)-y(9)) + (hA_th_hx*k_2*(y(7)-y(9))/mcp_h_hxfh)                              # T_cc1: y(9) 


# COOLANT-HELIUM HX
# fuel nodes
T_hch_c1 = W_c/m_c_hx*(y(4,t-tau_c_hx_f)-y(10)) + (hA_ct_hx*k_1*(y(12)-y(10))/mcp_h_c)                        # T_cf1: y(10)
T_hch_c2 = W_c/m_c_hx*(y(10)-y(11)) + (hA_ct_hx*k_2*(y(12)-y(11))/mcp_h_c)                                    # T_cf2: y(11)

# tubes
T_hch_t1 = (hA_ct_hx/mcp_t_hx)*((y(10)-y(12))+(y(11)-y(12)))+(hA_th_hx/mcp_t_hx)*((y(13)-y(12))+(y(14)-y(12)))   # T_ht1: y(12)

# helium
T_hch_h1 = W_hch_h1/m_h_hxfh*(y(16,t-tau_hx_c_c)-y(13)) + (hA_th_hx*k_1*(y(12)-y(13))/mcp_h_hxfh)                   # T_cc1: y(13) 
T_hch_h2 = W_hch_h2/m_h_hxfh*(y(13)-y(14)) + (hA_th_hx*k_2*(y(12)-y(14))/mcp_h_hxfh)                                # T_cc1: y(14) 

# HELIUM-WATER HX (FUEL LOOP)
# helium
T_hhwf_h1 = W_hhwf_h1/m_h_hxhw*(y(9,t-tau_c_hx_f)-y(15)) + (hA_ht_hxhw*(y(17)-y(15))/mcp_h_hxhw)                  # T_cf1: y(15)
T_hhwf_h2 = W_hhwf_h2/m_h_hxhw*(y(15)-y(16)) + (hA_ht_hxhw*(y(17)-y(16))/mcp_h_hxhw)                              # T_cf2: y(16)

# tubes
T_hhwf_t1 = (hA_ht_US_hxhw/mcp_t_hx)*((y(15)-y(17))+(y(16)-y(17)))+(hA_tw_hxhw/mcp_t_hx)*((y(18)-y(17))+(y(19)-y(17))) # T_ht1: y(17)

# water
T_hhwf_w1 = W_hhwf_w/m_w*(T0_hhwf_w1-y(18)) + (hA_tw_hxhw*k_1*(y(17)-y(18))/mcp_w)                                       # T_cc1: y(18) 
T_hhwf_w2 = W_hhwf_w/m_w*(y(18)-y(19)) + (hA_tw_hxhw*k_2*(y(17)-y(19))/mcp_w)                                            # T_cc1: y(19) 

# HELIUM-WATER HX (COOLANT LOOP)
# fuel nodes
T_hhwc_h1 = W_hhwc_h1/m_h_hxhw*(y(14,t-tau_c_hx_f)-y(20)) + (hA_ht_hxhw*k_1*(y(22)-y(20))/mcp_h_hxhw)                    # T_cf1: y(20)
T_hhwc_h2 = W_hhwc_h2/m_h_hxhw*(y(20)-y(21)) + (hA_ht_hxhw*k_2*(y(22)-y(21))/mcp_h_hxhw)                                 # T_cf2: y(21)

# tubes
T_hhwc_t1 = (hA_ht_hxhw/mcp_t_hx)*((y(20)-y(22))+(y(21)-y(22))) + (hA_tw_hxhw/mcp_t_hx)*((y(23)-y(22))+(y(24)-y(22))) # T_ht1: y(22)

# water
T_hhwc_w1 = W_hhwc_w/m_w*(T0_hhwf_w1-y(23)) + (hA_tw_hxhw*k_1*(y(22)-y(23))/mcp_w)                                       # T_cc1: y(23) maybe don't need these nodes
T_hhwc_w2 = W_hhwc_w/m_w*(y(23)-y(24)) + (hA_tw_hxhw*k_2*(y(22)-y(24))/mcp_w)                                            # T_cc1: y(24) 

n = (y(33)-beta_t)*y(25)/Lam+lam[0]*y(26)+lam[1]*y(27)+lam[2]*y(28)+lam[3]*y(29)+lam[4]*y(30)+lam[5]*y(31)                  # n (no source insertion): y(25)

# dC_i/dt (precursor concentrations)
C1 = y(25)*beta[0]/Lam-lam[0]*y(26)-y(26)/tau_c+y(13,t-tau_l)*np.exp(-lam[0]*tau_l)/tau_c                       # C1: y(26)
C2 = y(25)*beta[1]/Lam-lam[1]*y(27)-y(27)/tau_c+y(14,t-tau_l)*np.exp(-lam[1]*tau_l)/tau_c                       # C2: y(27)
C3 = y(25)*beta[2]/Lam-lam[2]*y(28)-y(28)/tau_c+y(15,t-tau_l)*np.exp(-lam[2]*tau_l)/tau_c                       # C3: y(28)
C4 = y(25)*beta[3]/Lam-lam[3]*y(29)-y(29)/tau_c+y(16,t-tau_l)*np.exp(-lam[3]*tau_l)/tau_c                       # C4: y(29)
C5 = y(25)*beta[4]/Lam-lam[4]*y(30)-y(30)/tau_c+y(17,t-tau_l)*np.exp(-lam[4]*tau_l)/tau_c                       # C5: y(30)
C6 = y(25)*beta[5]/Lam-lam[5]*y(31)-y(31)/tau_c+y(18,t-tau_l)*np.exp(-lam[5]*tau_l)/tau_c                       # C6: y(31)

# reactivity 
rho = (a_f/2)*(T_c_f1 + T_c_f2) # +  (a_b)*(T_c_b)           # rho: y(33)

Initial values & solve

In [ ]:
# instantiate jitcdde object
DDE = jitcdde([T_c_f1,T_c_f2,T_c_t1,T_c_c1,T_c_c2,T_hfh_f1,T_hfh_f2,T_hfh_t1,T_hfh_h1,T_hfh_h2,
               T_hch_c1,T_hch_c2,T_hch_t1,T_hch_h1,T_hch_h2,T_hhwf_h1,T_hhwf_h2,T_hhwf_t1,T_hhwf_w1,T_hhwf_w2,
               T_hhwc_h1,T_hhwc_h2,T_hhwc_t1,T_hhwc_w1,T_hhwc_w2,n,C1,C2,C3,C4,C5,C6,T_c_b,rho])

# set initial conditions
DDE.constant_past([T0_c_f1,T0_c_f2,T0_c_t1,T0_c_c1,T0_c_c2,T0_hfh_f1,T0_hfh_f2,T0_hfh_t1,T0_hfh_h1,T0_hfh_h2,
               T0_hch_c1,T0_hch_c2,T0_hch_t1,T0_hch_h1,T0_hch_h2,T0_hhwf_h1,T0_hhwf_h2,T0_hhwf_t1,T0_hhwf_w1,T0_hhwf_w2,
               T0_hhwc_h1,T0_hhwc_h2,T0_hhwc_t1,T0_hhwc_w1,T0_hhwc_w2,n_frac0,C0[0],C0[1],C0[2],C0[3],C0[4],C0[5],T0_c_b,0.0])

# DDE.step_on_discontinuities()

# jitcdde solver parameters 
t0 = 0.0
tf = 1000.00
T = np.arange(t0,tf,0.01)

sol_jit = []
for t_x in T:
    sol_jit.append(DDE.integrate(t_x))

In [ ]:
fig,axs = plt.subplots(3,3,figsize=(18,18))

t_0 = 0.0
t_f = 75.0

axs[0,0].set_xlim([t_0,t_f])
axs[0,0].plot(T,[s[0] for s in sol_jit],label="core f1") 
axs[0,0].plot(T,[s[1] for s in sol_jit],label="core f2") 
axs[0,0].plot(T,[s[2] for s in sol_jit],label="core t1")  
axs[0,0].plot(T,[s[3] for s in sol_jit],label="core c1")
axs[0,0].plot(T,[s[4] for s in sol_jit],label="core c2")
axs[0,0].legend()
axs[0,0].set_title("Core Node Temperatures (K)")
axs[0,0].tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off


axs[0,1].set_xlim([t_0,t_f])
# axs[0,1].set_ylim([650,750])
axs[0,1].plot(T,[s[5] for s in sol_jit],label="hx_fh f1") 
axs[0,1].plot(T,[s[6] for s in sol_jit],label="hx_fh f2") 
axs[0,1].plot(T,[s[7] for s in sol_jit],label="hx_fh t1")  
#axs[0,1].plot(T,[s[8] for s in sol_jit],label="hx_fh h1")
#axs[0,1].plot(T,[s[9] for s in sol_jit],label="hx_fh h2")
axs[0,1].legend()
axs[0,1].set_title("HX Fuel->Helium Node Temperatures (K)")
#axs[0,1].tick_params(
#    axis='x',          # changes apply to the x-axis
#    which='both',      # both major and minor ticks are affected
#    bottom=False,      # ticks along the bottom edge are off
#    top=False,         # ticks along the top edge are off
#    labelbottom=False) # labels along the bottom edge are off

# fuel temps
axs[0,2].set_xlim([t_0,t_f])
axs[0,2].plot(T,[s[15] for s in sol_jit],label="hx_hwf h1") 
axs[0,2].plot(T,[s[16] for s in sol_jit],label="hx_hwf h2") 
axs[0,2].plot(T,[s[17] for s in sol_jit],label="hx_hwf t1")  
axs[0,2].plot(T,[s[18] for s in sol_jit],label="hx_hwf w1")
axs[0,2].plot(T,[s[19] for s in sol_jit],label="hx_hwf w2")
axs[0,2].legend()
axs[0,2].set_title("HX Helium->Water (Fuel Loop) Node Temperatures (K)")
axs[0,2].tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off

# fuel temps
axs[1,1].set_xlim([t_0,t_f])
axs[1,1].plot(T,[s[10] for s in sol_jit],label="hx_ch f1") 
axs[1,1].plot(T,[s[11] for s in sol_jit],label="hx_ch f2") 
axs[1,1].plot(T,[s[12] for s in sol_jit],label="hx_ch t1")  
axs[1,1].plot(T,[s[13] for s in sol_jit],label="hx_ch h1")
axs[1,1].plot(T,[s[14] for s in sol_jit],label="hx_ch h2")
axs[1,1].legend()
axs[1,1].set_title("HX Coolant->Helium Node Temperatures (K)")

# fuel temps
axs[1,2].set_xlim([t_0,t_f])
axs[1,2].plot(T,[s[20] for s in sol_jit],label="hx_hwc h1") 
axs[1,2].plot(T,[s[21] for s in sol_jit],label="hx_hwc h2") 
axs[1,2].plot(T,[s[22] for s in sol_jit],label="hx_hwc t1")  
axs[1,2].plot(T,[s[23] for s in sol_jit],label="hx_hwc w1")
axs[1,2].plot(T,[s[24] for s in sol_jit],label="hx_hwc w2")
axs[1,2].legend()
axs[1,2].set_title("HX Helium->Water (Coolant Loop) Node Temperatures (K)")

axs[1,0].plot(T,[s[25] for s in sol_jit],label="n") 
axs[1,0].set_xlabel("t (s)")
axs[1,0].set_title(r"$n$")
axs[1,0].set_ylabel(r"$\frac{n}{n_0}$")
axs[1,0].set_xlim([t_0,t_f])

axs[2,0].plot(T,[s[33] for s in sol_jit],label="n") 
axs[2,0].set_xlabel("t (s)")
axs[2,0].set_title(r"$\rho$")
axs[2,0].set_xlim([t_0,t_f])

axs[2,1].plot(T,[s[26] for s in sol_jit],label="C1") 
axs[2,1].plot(T,[s[27] for s in sol_jit],label="C2") 
axs[2,1].plot(T,[s[28] for s in sol_jit],label="C3")  
axs[2,1].plot(T,[s[29] for s in sol_jit],label="C4")
axs[2,1].plot(T,[s[30] for s in sol_jit],label="C5")
axs[2,1].plot(T,[s[31] for s in sol_jit],label="C6")
axs[2,1].legend()
axs[2,1].set_xlabel("t (s)")
axs[2,1].set_yscale("log")
axs[2,1].set_ylabel(r"concentration (1/cm$^3$)")
axs[2,1].legend(loc="right")
axs[2,1].set_title("Precursor Concentrations")
axs[2,1].set_xlim([t_0,t_f])